# Topic modelling on news data 25 topics

- Data is taken from kaggle.
- Applying the topic modeling the see the results for 25 topics
- And choose the results for 2^nd level topic modeling

In [1]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv',error_bad_lines=False)

In [2]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
print(len(documents))
print(documents[:5])

1103665
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


# Data preprocessing 
## Different process we do here like as Tokenization ,lemetization and stemming the data
- we wanted to convert the data to their normal form for example 'stolen' would converted to steal
### Here nltk is used for removing different language rather than english (Hindi and urdu like that word would be removed)
- In preprocessing actually we remove all the punctuation marks , exclamatory marks and commas 

In [4]:

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
               
            

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Lemmatization 
## Lemmatization is used for gouping of word that's contains the same meaning(synonyms,antonyms)
# Tokenization
## Tokenization is used for keeps the word having meaningfull meaning
- This is used for removal of word like if,the ,a,an that word doesn't make any sense in Topic
# Stemming
## Stemming is used for convert the word into their root form

In [5]:
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

# preview data after preprocessing 
- How the data will look like 

In [6]:
# Select a document to preview after preprocessing
doc_sample = documents[documents['index'] == 4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))



original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfires']


In [7]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0              [decide, community, broadcast, licence]
1                         [witness, aware, defamation]
2           [call, infrastructure, protection, summit]
3                          [staff, aust, strike, rise]
4             [strike, affect, australian, travellers]
5               [ambitious, olsson, win, triple, jump]
6               [antic, delight, record, break, barca]
7    [aussie, qualifier, stosur, waste, memphis, ma...
8             [aust, address, security, council, iraq]
9                         [australia, lock, timetable]
Name: headline_text, dtype: object

### Dictionary is formed for 

In [8]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 community
2 decide
3 licence
4 aware
5 defamation
6 witness
7 call
8 infrastructure
9 protection
10 summit


## Filtering the word

In [9]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(112, 1), (489, 1), (970, 1), (4427, 1)]

In [11]:
import pickle
pickle.dump(bow_corpus, open('bow_corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [12]:
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 112 ("help") appears 1 time.
Word 489 ("rain") appears 1 time.
Word 970 ("bushfires") appears 1 time.
Word 4427 ("dampen") appears 1 time.


In [13]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5955574839932255),
 (1, 0.394834368882524),
 (2, 0.4976240783705052),
 (3, 0.4917188028077664)]


In [14]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=25, id2word=dictionary, passes=2, workers=2)
lda_model.save('model25.gensim')

In [15]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.074*"warn" + 0.064*"jail" + 0.046*"life" + 0.041*"sentence" + 0.041*"lead" + 0.030*"case" + 0.029*"speak" + 0.028*"mark" + 0.026*"issue" + 0.024*"increase"
Topic: 1 
Words: 0.049*"interview" + 0.048*"country" + 0.040*"years" + 0.036*"child" + 0.035*"time" + 0.031*"trial" + 0.028*"guilty" + 0.027*"talk" + 0.026*"children" + 0.026*"community"
Topic: 2 
Words: 0.037*"beat" + 0.033*"drum" + 0.030*"aboriginal" + 0.029*"mine" + 0.028*"show" + 0.027*"project" + 0.027*"study" + 0.027*"company" + 0.023*"cancer" + 0.019*"international"
Topic: 3 
Words: 0.112*"australian" + 0.054*"year" + 0.046*"win" + 0.045*"china" + 0.036*"dead" + 0.027*"head" + 0.024*"strike" + 0.017*"unite" + 0.016*"kid" + 0.016*"despite"
Topic: 4 
Words: 0.072*"world" + 0.052*"state" + 0.041*"work" + 0.040*"minister" + 0.039*"deal" + 0.034*"tell" + 0.028*"farmers" + 0.024*"trade" + 0.022*"free" + 0.021*"sign"
Topic: 5 
Words: 0.058*"plan" + 0.049*"council" + 0.044*"call" + 0.041*"water" + 0.038*"national" 

In [16]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfires']

In [17]:
unseen_document = 'How a Pentagon de to help him identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Score: 0.3879478871822357	 Topic: 0.150*"say" + 0.052*"canberra" + 0.050*"rural" + 0.035*"people" + 0.031*"need" + 0.024*"young" + 0.022*"energy" + 0.020*"rugby" + 0.019*"doctor" + 0.015*"victims"
Score: 0.2280394583940506	 Topic: 0.051*"high" + 0.047*"league" + 0.047*"lose" + 0.038*"island" + 0.033*"race" + 0.032*"future" + 0.027*"challenge" + 0.027*"road" + 0.023*"club" + 0.023*"safety"
Score: 0.2079799920320511	 Topic: 0.064*"report" + 0.064*"school" + 0.033*"victoria" + 0.033*"close" + 0.030*"darwin" + 0.026*"week" + 0.026*"inquiry" + 0.023*"president" + 0.023*"go" + 0.022*"festival"


In [18]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('bow_corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model25.gensim')


In [19]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


## As topic related to health is syncronised well so we increase the number of topic stop here.
